In [66]:
import os
import math
import datetime
import pandas as pd

def calculate_h_index(quotation): #h_ndex
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    h_index = 0
    for i, quotation in enumerate(quotation):
        if quotation >= i + 1:
            h_index = i + 1
        else:
            break

    return h_index

def calculate_i10_index(quotation):  #i10_index
    i10_index = sum(1 for quotation in quotation if quotation >= 10)
    return i10_index

def calculate_g_index(quotation):  #g_index
    quotation.sort(reverse=True)  # 논문을 내림차순으로 정렬

    g_index = 0
    quotation_sum=0
    for i, quotation in enumerate(quotation):
        quotation_sum+=quotation
        if quotation_sum >= (i + 1)**2:
            g_index = i + 1

    return g_index

#Object 타입에서 숫자형 체크용
def is_numeric(value):
    try:
        float(value)  # float으로 변환 가능한 경우 숫자
        return True
    except (ValueError, TypeError):
        return False

def recentness(pYears):
    # dt = datetime.datetime.now()#현재날짜
    dt = 2013
    rct = 0
    lct = 0
    if len(pYears) == 0:
        rct = 0
        lct = 0
    else:
        year_avg = sum(pYears) / len(pYears)
        if year_avg >= int(dt)-2: #int(dt.year)-2:
            lct = 1
        # else:
        #     lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)
        elif int(dt)-15 < year_avg <= int(dt)-3:
            lct = max(round((1-(int(dt)-3-year_avg)*0.1),2), 0)

        rct += len(pYears) #연도 개수

    return rct, lct

In [ ]:


max_scoquality = 62888.30906843457
max_rct = 15106

max_lct = []

df_a_copy = pd.DataFrame()

# df_a_copy의 각 행에서 1번 열의 값을 추출
for index, row in df_a_copy.iterrows(): # 저자로 교체
    name = row[5]  # 1번 열의 값

    # 각 줄을 나누어서 리스트로 저장
    line_lists = [line.strip().split('|') for line in lines] # 몽고디비 데이터로 교체, 또는 삭제
    
    #빈 리스트 생성
    citations = [] #인용수
    years = [] #연도
    scoquality = 0 #품질
    for line_list in line_lists: #논문으로 교체
        if len(line_list) > 0:
            citations.append(int(line_list[0]))
            years.append(int(line_list[1]))
            scoquality += math.log(int(line_list[0]*1)+1)
    
    # print(name, years)
    rct, lct = recentness(years)
    # max_lct.append(rct)
    norm_scoquality = scoquality/max_scoquality
    # norm_recentness = rct/max_rct*0.66*lct
    norm_recentness = ((rct/max_rct)+lct)*0.66
    score = (norm_scoquality + norm_recentness)*50
    # print(rct, scoquality, scoquality/max_scoquality, rct/max_rct, lct, norm_recentness, score)
    #각 index 계산 후 df_a_copy에 새로운 열로 추가
    df_a_copy.at[index,"Paper Count"] = len(citations)
    df_a_copy.at[index,"h-index"] = calculate_h_index(citations)
    df_a_copy.at[index,"i10-index"] = calculate_i10_index(citations)
    df_a_copy.at[index,"g-index"] = calculate_g_index(citations)
    df_a_copy.at[index,"scoquality"] = norm_scoquality
    df_a_copy.at[index,"recentness"] = norm_recentness
    df_a_copy.at[index,"score"] = score
        
df_a_copy['ID'] = 'id_' + df_a_copy['ID']

# print(df_a_copy)
df_a_copy.head(20)
# df_a_copy.to_csv('output.txt', sep = ';', index=False)
# df_a_copy.to_csv('output_index.txt', index=True)
df_a_copy.to_csv('output.csv', index=False)
df_a_copy.to_csv('output_index.csv', index=True)